## BCAC preprocessing

### Convert imputed oxford files to bed

#### Reformat the raw sample files of oncoarray (and iCOGS?)

In [ ]:
eth="euro"
array="oncoarray"
echo "ID_1 ID_2 missing sex" > elkon_723_${eth}_${array}_sample_order.sample
echo "0 0 0 D" >> elkon_723_${eth}_${array}_sample_order.sample
cat elkon_723_${eth}_${array}_sample_order.txt | awk '{print $1" "$1" 0 2"}' >> elkon_723_${eth}_${array}_sample_order.sample

#### For the Hispanic dataset, run the following alternative block

In [ ]:
eth="hispanic"
echo "ID_1 ID_2 missing sex" > elkon_723_${eth}_sample_order.sample
echo "0 0 0 D" >> elkon_723_${eth}_sample_order.sample
cat elkon_723_${eth}_sample_order.txt | awk '{print $2" "$2" 0 2"}' >> elkon_723_${eth}_sample_order.sample

#### iCOGS dosage files needs to be reformatted too

In [ ]:
gunzip -c  elkon_723_euro_icogs_imputed_probs_chr2.txt.gz | sed 's/^/--- /' | gzip >  elkon_723_euro_icogs_imputed_probs_chr2.fixed2.txt.gz

In [ ]:
 for a in {10..15}; do plink --gen elkon_723_${eth}_${array}_imputed_probs_chr${a}.fixed2.txt.gz --sample elkon_723_${eth}_${array}_sample_order.sample --oxford-single-chr ${a} --make-bed --threads 50 --out ${target_path_elkon}/raw/ox_to_bed/ds${a}; done

#### Convert raw (oxford/gen) files to bed files

In [ ]:
target="bcac_onco_aj"
eth="euro"
array="oncoarray"
imp="impX"
target_path_elkon="/specific/elkon/hagailevi/PRS/datasets/dec/"${target}"/${imp}/"
mkdir -p ${target_path_elkon}/raw/ox_to_bed || echo ""
for a in {22..22}; do plink --gen elkon_723_${eth}_${array}_imputed_probs_chr${a}.txt.gz --sample elkon_723_${eth}_${array}_sample_order.sample --oxford-single-chr ${a} --keep pop.il.panel --make-bed --threads 50 --out ${target_path_elkon}/raw/ox_to_bed/ds${a}; done


In [ ]:
target="bcac_onco_eur"
eth="euro"
array="oncoarray"
imp="impX"
target_path_elkon="/specific/elkon/hagailevi/PRS/datasets/dec/"${target}"/${imp}/"
mkdir -p ${target_path_elkon}/raw/ox_to_bed || echo ""
for a in {16..22}; do plink --gen elkon_723_${eth}_${array}_imputed_probs_chr${a}.fixed2.txt.gz --sample elkon_723_${eth}_${array}_sample_order.sample --oxford-single-chr ${a} --make-bed --threads 50 --out ${target_path_elkon}/raw/ox_to_bed/ds${a}; done


### Create a subset of of the 313 SNPs 

In [ ]:
plink --bfile ds0 --extract $PRS_GWASS/bca_313/313.valid.snp --a1-allele $PRS_GWASS/bca_313/313_rsids.tsv 5 3 --make-bed --out ds

### Generate GWAS from BCAC data using plink2 with PCs (TBD!)

In [ ]:
eth=pl
target="bcac_onco_eur-minus-${eth}"
imp="impX_new"

if [[ ! -d $PRS_GWASS/${target} ]]
    echo "Creating dir: $PRS_GWASS/${target}"
    mkdir -p $PRS_GWASS/${target}
fi

plink2 --bfile ds.QC --glm --pheno $PRS_DATASETS/${target}/${imp}/pheno --allow-no-sex --covar ds...TBD! --covar-name PC1,PC2,PC3,PC4,PC5 --out $PRS_GWASS/${target}/gwas
#plink2 --bfile ds.QC --glm --pheno $PRS_DATASETS/${target}/${imp}/pheno --allow-no-sex --covar covar_reformatted --covar-name PC1,PC2,PC3,PC4,PC5,country,history,age,er,brca12 --out gwas_covar

#### Modify BCAC's SNP id file to correct format (using vim)

In [ ]:
:   1,$s/\:*\:.\{-}\t/\t
:   g!/rs/d